In [2]:
import pandas as pd
import quandl, math
import numpy as np
from sklearn import preprocessing,cross_validation, svm
from sklearn.linear_model import LinearRegression

F:\InstallationFiles\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
data = pd.read_csv("data.csv")
df = data [ ['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume',] ]
df[ 'HL_PCT' ] = (df['Adj. High'] - df['Adj. Close']) / df['Adj. Close'] * 100.0
df[ 'PCT_change' ] = (df['Adj. Close'] - df['Adj. Open'] ) / df['Adj. Open'] * 100.0
#define a new data frame
df = df[ [ 'Adj. Close' , 'HL_PCT' , 'PCT_change', 'Adj. Volume' ] ]

F:\InstallationFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
F:\InstallationFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [4]:
########### FIX THE DATABASE
forecast_col = 'Adj. Close'
df.fillna(-99999, inplace= True )
forecast_out = int ( math.ceil( 0.01 * len(df) ) )  #no. of days to forecast

print ( "no of days forecasted ", forecast_out )

no of days forecasted  35


In [5]:
df ['label'] = df[forecast_col].shift(-forecast_out)
print (df.head() )
print (df.tail() )

   Adj. Close    HL_PCT  PCT_change  Adj. Volume      label
0   50.322842  3.712563    0.324968   44659000.0  69.078238
1   54.322689  0.710922    7.227007   22834300.0  67.839414
2   54.869377  3.729433   -1.227880   18256100.0  68.912727
3   52.597363  6.417469   -5.726357   15247300.0  70.668146
4   53.164113  1.886792    1.183658    9188600.0  71.219849
      Adj. Close    HL_PCT  PCT_change  Adj. Volume  label
3419     1094.00  1.343693    0.130884    1990515.0    NaN
3420     1053.15  2.921711   -2.487014    3418154.0    NaN
3421     1026.55  3.918952   -2.360729    2413517.0    NaN
3422     1054.09  0.491419    0.332191    3272409.0    NaN
3423     1006.94  5.720301   -5.353887    2940957.0    NaN


In [6]:
df.dropna( inplace = True )
print (df.tail() )

      Adj. Close    HL_PCT  PCT_change  Adj. Volume    label
3384     1177.37  0.896914   -0.029718    1792602.0  1094.00
3385     1182.22  0.346805   -0.134312    1643877.0  1053.15
3386     1181.59  0.495942    0.476195    2774967.0  1026.55
3387     1119.20  1.081129   -0.729098    5798880.0  1054.09
3388     1068.76  4.325574   -2.893850    3742469.0  1006.94


In [7]:
X = np.array( df.drop(['label'], 1) )
y = np.array ( df['label'] )
X = preprocessing.scale(X)
df.dropna(inplace = True )
y = np.array( df['label'] )

In [8]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split ( X, y, test_size = 0.2 )

clf = LinearRegression(n_jobs = -1)
clf.fit( X_train, y_train )
accuracy = clf.score(X_test, y_test )

print ( accuracy) 

0.9788528516332876


In [9]:
from sklearn import ensemble
clf = ensemble.GradientBoostingRegressor(n_estimators = 400, max_depth = 5, min_samples_split = 2,
          learning_rate = 0.1, loss = 'ls')
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.9844721681674757

In [10]:
from sklearn import svm
model = svm.SVR(kernel='rbf', C=100, gamma=0.0001)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))

0.2196347513699305


In [11]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(max_depth=4, min_samples_split=2, max_leaf_nodes=17, min_samples_leaf=1)
# fit model
model.fit(X_train, y_train)
print(model.score(X_test, y_test))

0.9827564680737794


In [12]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(max_features=4, n_estimators=500, bootstrap=False)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))

0.9735858240315458


In [13]:
from sklearn.neural_network import MLPRegressor
mlpreg = MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu',solver='adam', alpha=0.01, learning_rate_init=0.0001, max_iter=500)
mlpreg.fit(X_train, y_train)
print(mlpreg.score(X_test, y_test))

0.9295324029113281


F:\InstallationFiles\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
